In [1]:
import pandas as pd
import numpy as np
from glob import glob
from tqdm.notebook import tqdm

BASE = "/media/giani/Gianicosas/Magister/Proyecto/raw_data"

# Operacionalización de datos

### Alumno desertor
- Tiene que haber pasado un tiempo mayor a la duración de la carrera (`cursando==0`)
- Considerar todo para el mismo codigo de carrera cada vez
- No debe estar en la base de titulados para esa carrera

### Tipos de deserción

- Sale del sistema y no vuelve a ingresar
- Sale del sistema y vuelve después de 1, 2 ó 3 años:
    - Vuelve a la misma carrera en la misma institución
    - Vuelve a otra carrera en la misma institución
    - Vuelve a la misma carrera en otra institución

# Matriculados

### Todas las columnas
```python
# pd.read_csv(f"{BASE}/20220719_Matrícula_Ed_Superior_2022_PUBL_MRUN.csv", nrows=5, sep=";").columns
[
    'cat_periodo', 'id', 'codigo_unico', 'mrun', 'gen_alu', 'fec_nac_alu', 'rango_edad', 'anio_ing_carr_ori',
    'sem_ing_carr_ori', 'anio_ing_carr_act', 'sem_ing_carr_act', 'anio_ing_carr_act', 'tipo_inst_1', 'tipo_inst_2', 'tipo_inst_3',
    'cod_inst', 'nomb_inst', 'cod_sede', 'nomb_sede', 'cod_carrera', 'nomb_carrera', 'modalidad', 'jornada',
    'version', 'tipo_plan_carr', 'dur_estudio_carr', 'dur_proceso_tit', 'dur_total_carr', 'region_sede',
    'provincia_sede', 'comuna_sede', 'nivel_global', 'nivel_carrera_1', 'nivel_carrera_2', 'requisito_ingreso',
    'vigencia_carrera', 'formato_valores', 'valor_matricula', 'valor_arancel', 'codigo_demre', 'area_conocimiento',
    'cine_f_97_area', 'cine_f_97_subarea', 'area_carrera_generica', 'cine_f_13_area', 'cine_f_13_subarea',
    'acreditada_carr', 'acreditada_inst', 'acre_inst_desde_hasta', 'acre_inst_anio', 'costo_proceso_titulacion',
    'costo_obtencion_titulo_diploma', 'forma_ingreso'
]
```

### Cols para determinar vector objetivo
```python
['cat_periodo', 'codigo_unico', 'mrun', 'anio_ing_carr_ori', 'anio_ing_carr_act', 'tipo_inst_1', 'dur_estudio_carr']
```

### Tipos de instituciones
```python
# Salida: ['Universidades', 'Institutos Profesionales', 'Centros de Formación Técnica']
pd.concat([
    pd.read_csv(f, sep=";", usecols=["tipo_inst_1"])
    for f in tqdm(glob(f"{BASE}/20220719_Matrícula_Ed_Superior_*.csv"), total=16)
])["tipo_inst_1"].unique()
instituciones = ['Institutos Profesionales', 'Centros de Formación Técnica']
```

## Lectura de datos matriculados

Fuente: https://datosabiertos.mineduc.cl/matricula-en-educacion-superior/

- Se considera solo matriculados en IP y CFT
- Se elimina los registros con `mrun` nulo
- Se considera solo registros desde el 2015
- Se reemplaza los datos `9995` por `np.nan`

In [2]:
def iter_df_read(file, col_name, condition_list, chunksize=1000, **kwargs):
    iter_csv = pd.read_csv(file, iterator=True, chunksize=chunksize, **kwargs)
    return pd.concat([chunk[chunk[col_name].isin(condition_list)] for chunk in iter_csv])

In [3]:
cols_time = [
    'cat_periodo', 'codigo_unico', 'mrun', 'anio_ing_carr_ori', 'anio_ing_carr_act',
    'tipo_inst_1', 'dur_estudio_carr'
]
instituciones = ['Institutos Profesionales', 'Centros de Formación Técnica']

matriculados = pd.concat([
    iter_df_read(f, "tipo_inst_1", instituciones, sep=";", usecols=cols_time)
    for f in tqdm(
        glob(f"{BASE}/post_2015/20220719_Matrícula_Ed_Superior_*.csv"),
        total=8
    )
]).dropna(subset="mrun")

matriculados.loc[:, ['anio_ing_carr_ori', 'anio_ing_carr_act']] = matriculados[
    ['anio_ing_carr_ori', 'anio_ing_carr_act']
].replace(
    [9995], [np.nan] 
)

print(f"Cantidad de registros: {matriculados.shape[0]}\n")
print(f"Cantidad de nulos: \n{matriculados.isna().sum()}")
matriculados.head()

  0%|          | 0/8 [00:00<?, ?it/s]

Cantidad de registros: 4131609

Cantidad de nulos: 
cat_periodo                0
codigo_unico               0
mrun                       0
anio_ing_carr_ori          0
anio_ing_carr_act    1193433
tipo_inst_1                0
dur_estudio_carr           0
dtype: int64


,cat_periodo,codigo_unico,mrun,anio_ing_carr_ori,anio_ing_carr_act,tipo_inst_1,dur_estudio_carr
0,2015,I374S1C9J1V1,5.0,2015,NaN,Centros de Formación Técnica,5
1,2015,I498S6C132J2V1,37.0,2015,NaN,Centros de Formación Técnica,5
5,2015,I111S12C104J1V1,107.0,2013,NaN,Institutos Profesionales,9
7,2015,I111S14C309J2V1,118.0,2015,NaN,Institutos Profesionales,5
9,2015,I260S44C120J2V1,184.0,2014,NaN,Centros de Formación Técnica,5


## Pre procesamiento: Año ingreso carrera actual

En los años anteriores a 2018 este dato está nulo en muchos registros. Por ahora se reemplaza el valor por el del año de ingreso a carrera original.
- **TODO**: Tratamiento de datos 9998, 9999

In [7]:
matriculados['anio_ing_carr_act'] = matriculados['anio_ing_carr_act'].fillna(matriculados['anio_ing_carr_ori'])
matriculados['anio_ing_carr_act'].isna().sum()

0

In [8]:
matriculados['anio_ing_carr_act'].isin([9998, 9999]).value_counts()

False    4131263
True         346
Name: anio_ing_carr_act, dtype: int64

In [9]:
matriculados = matriculados[~matriculados['anio_ing_carr_act'].isin([9998, 9999])]

### Pre procesamiento: Cursando carrera

Se crea columna `cursando`, la cual indica:
- `1` si los años transcurridos desde el año de ingreso a la carrera hasta el año 2022 es mayor a la duración de la carrera en años.
- `0` en caso contrario

Esto se utilizará más adelante en la construcción del vector objetivo; Se hace una selección de datos donde solo se selecciona a aquellos que tengan `cursando==0`, para medir inicialmente deserción solamente en aquellos que no se encuentren ya en la ventana de duración de la carrera.

In [12]:
matriculados = matriculados.dropna(subset=["mrun"]).reset_index(drop=True)
matriculados["cursando"] = np.where(
    pd.Series([2022] * matriculados.shape[0]) - matriculados["anio_ing_carr_act"]
    > matriculados["dur_estudio_carr"] / 2, 0, 1
)

matriculados["cursando"].value_counts("%")

0    0.666067
1    0.333933
Name: cursando, dtype: float64

In [14]:
# Algunos aparecen 2 veces probablemente porque se matriculan 2 veces al año, por eso se elimina duplicados
termino_duracion_carrera = matriculados[matriculados["cursando"]==0][
    ["cat_periodo", "codigo_unico", "mrun", "anio_ing_carr_act"]
].drop_duplicates().reset_index(drop=True)

print(
    f"Cantidad de registros de registros con alumnos en ventana "
    f"de duración de carrera: {termino_duracion_carrera.shape[0]}"
)

Cantidad de registros de registros con alumnos en ventana de duración de carrera: 2751462


# Titulados

Fuente: https://datosabiertos.mineduc.cl/titulados-en-educacion-superior/

### Todas las columnas
```python
[
    'cat_periodo', 'codigo_unico', 'mrun', 'gen_alu', 'fec_nac_alu', 'rango_edad', 'anio_ing_carr_ori',
    'sem_ing_carr_ori', 'anio_ing_carr_act', 'sem_ing_carr_act', 'nomb_titulo_obtenido',
    'nomb_grado_obtenido', 'fecha_obtencion_titulo', 'tipo_inst_1', 'tipo_inst_2', 'tipo_inst_3',
    'cod_inst', 'nomb_inst', 'cod_sede', 'nomb_sede', 'cod_carrera', 'nomb_carrera', 'nivel_global',
    'nivel_carrera_1', 'nivel_carrera_2', 'dur_estudio_carr', 'dur_proceso_tit', 'dur_total_carr',
    'region_sede', 'provincia_sede', 'comuna_sede', 'jornada', 'modalidad', 'version', 'tipo_plan_carr',
    'area_cineunesco', 'area_cine_f_97', 'subarea_cine_f_97', 'area_cine_f_13', 'subarea_cine_f_13',
    'area_carrera_generica_n'
]
```

### Cols para determinar vector objetivo
```python
['cat_periodo', 'codigo_unico', 'mrun', 'tipo_inst_1', 'fecha_obtencion_titulo']
```

In [21]:
cols_tit = ['cat_periodo', 'codigo_unico', 'mrun', 'tipo_inst_1', 'fecha_obtencion_titulo']

titulados = pd.concat([
    iter_df_read(f, "tipo_inst_1", instituciones, sep=";", usecols=cols_tit)
    for f in tqdm(
        glob(f"{BASE}/titulados_post_2015/20220804_Titulados_Ed_Superior_*.csv"),
        total=7
    )
]).dropna(subset="mrun")

  0%|          | 0/7 [00:00<?, ?it/s]

In [22]:
print(titulados.shape)
titulados.head()

(748517, 5)


,cat_periodo,codigo_unico,mrun,fecha_obtencion_titulo,tipo_inst_1
2,2015,I111S12C87J1V1,249.0,20160106,Institutos Profesionales
7,2015,I260S8C79J1V1,1155.0,20151116,Centros de Formación Técnica
8,2015,I126S1C273J2V1,1202.0,20150909,Institutos Profesionales
9,2015,I111S9C2J2V1,1317.0,20150430,Institutos Profesionales
12,2015,I117S1C27J4V1,1619.0,20160208,Institutos Profesionales


# Unión de bases

In [31]:
merge = pd.merge(
    termino_duracion_carrera,
    titulados[["cat_periodo", "codigo_unico", "mrun", "fecha_obtencion_titulo"]],
    how="left"
)

termino_duracion_carrera.shape, titulados.shape, merge.shape

((2751462, 4), (748517, 5), (2751465, 5))

In [41]:
merge[
    ["codigo_unico", "mrun", "fecha_obtencion_titulo"]
].drop_duplicates()[
    "fecha_obtencion_titulo"
].isna().value_counts("%").rename(
    index={True: "Desertor", False: "Titulado"}
) * 100

Desertor    76.937768
Titulado    23.062232
Name: fecha_obtencion_titulo, dtype: float64